# Modelos de Prediccion de Demanda

## Índice

- [Librerias](#Librerias)
- [Manipulacion de Datos](#Manipulacion-de-Datos)
- [Importar Datos](#Importar-Datos)
- [Analisis Descriptivo](#Analisis-Descriptivo)
- [Seleccion de Data](#Seleccion-de-Data)
    - [Obtencion de datos de entrenamiento](#Obtencion-de-datos-de-entrenamiento)
    - [Obtencion de datos de validacion](#Obtencion-de-datos-de-validacion)
    - [Obtencion de datos de prueba](#Obtencion-de-datos-de-prueba)
- [Analisis de Datos](#Analisis-de-datos)
    - [Correlacion General](#Correlacion-General)
    - [Correlacion Especifica de Demanda](#Correlacion-Especifica-de-Demanda)
- [Metricas de Evaluacion](#Metricas-de-Evaluacion)
    - [KPI](#KPI)
    - [Error Absoluto](#Error-Absoluto)
- [Creacion de Modelos](#Creacion-de-Modelos)
    - [Creacion de matrices](#Creacion-de-matrices)
        
    - [Entrenamiento de Modelos](#Entrenamiento-de-Modelos)
- [Informe de Modelos](#Informe-de-Modelos)
    - [Todas las variables](#Todas-las-variables)

# Librerias

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
from tabulate import tabulate
from datetime import datetime
import matplotlib.pyplot as ax
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from sklearn.metrics import r2_score, max_error, mean_squared_error, mean_absolute_error

# Funciones

scaler = StandardScaler()

class Transformar:
    def __init__(self, matriz):
        self.matriz = np.array(matriz)

    def normalizar(self):
        # Reshape the data if it is one-dimensional
        if len(self.matriz.shape) == 1:
            self.matriz = self.matriz.reshape(-1, 1)
        self.matriz_normalizada = scaler.fit_transform(self.matriz)
        return self.matriz_normalizada

    def desnormalizar(self):
        if len(self.matriz.shape) == 1:
            self.matriz = self.matriz.reshape(-1, 1)
        self.matriz_desnormalizada = scaler.inverse_transform(self.matriz).flatten()
        return self.matriz_desnormalizada
    
    def parametros(self):
        return scaler.mean , scaler.scale

# Manipulacion de Datos

In [ ]:
data = pd.read_csv(r"data/dataset.csv")
df = pd.DataFrame(data)

df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Year'] = df['Fecha'].dt.year.astype(int)
df['Month'] = df['Fecha'].dt.month.astype(int)
df['Month Day'] = df['Fecha'].dt.day.astype(int)
df['Weekday'] = df['Fecha'].dt.weekday.astype(int)
df['Yearday'] = df['Fecha'].dt.dayofyear.astype(int)
df = df.sort_values(by='Fecha')
df

## Analisis Descriptivo

In [ ]:
print("\nTabla Cabezera:")
print(tabulate(df.head(), headers='keys', tablefmt='pretty'))

print("\nTabla Descriptiva:")
print(tabulate(df.describe(), headers='keys', tablefmt='pretty'))

print("\nTabla Datos Duplicados:")
duplicados = df.duplicated()
print(tabulate(df[duplicados], headers='keys', tablefmt='pretty'))

print("\nTabla de Informacion:")
print(tabulate(df.info(), headers='keys', tablefmt='pretty'))

#### Creacion de matrices de entrenamiento, validacion y prueba

## Analisis de Datos

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20, 10))
plt.rcParams.update({'font.size': 23})

tiempo = np.arange(0,len(Demanda_t0),1)/24

# Set axis ranges; by default this will put major ticks every 25.
ax.set_xlim(-100,1850)
ax.set_ylim(0,13000)

# Change major ticks to show every 20.
ax.xaxis.set_major_locator(MultipleLocator(200))
ax.yaxis.set_major_locator(MultipleLocator(1000))

# Change minor ticks to show every 5. (20/4 = 5)
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))

# Turn grid on for both major and minor ticks and style minor slightly
# differently.
ax.grid(which='major', color='#CCCCCC', linestyle='--')
ax.grid(which='minor', color='#CCCCCC', linestyle=':')

ax.plot(tiempo,Demanda_t0,color='b')
ax.set_xlabel('Tiempo [Horas]\n' r'$(a)$')
ax.set_ylabel('Demanda [MW]')
# ax.set_title('ARIMAX prediccion de tiempo de 15 minutos',style = "italic",fontsize=19)
ax.legend([r'$ P_{real} $',r'$ P_{prediccion} $'],ncol=2,loc='upper right',fontsize=15,fancybox=True, framealpha=0.5)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(20, 10))
plt.rcParams.update({'font.size': 23})

tiempo = np.arange(0,len(Demanda_t0),1)/24

# Set axis ranges; by default this will put major ticks every 25.
ax.set_xlim(0,168)
ax.set_ylim(0,13000)

# Change major ticks to show every 20.
ax.xaxis.set_major_locator(MultipleLocator(20))
ax.yaxis.set_major_locator(MultipleLocator(1000))

# Change minor ticks to show every 5. (20/4 = 5)
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))

# Turn grid on for both major and minor ticks and style minor slightly
# differently.
ax.grid(which='major', color='#CCCCCC', linestyle='--')
ax.grid(which='minor', color='#CCCCCC', linestyle=':')

ax.plot(tiempo,Demanda_t0,color='b')
ax.set_xlabel('Tiempo [Horas]\n')
ax.set_ylabel('Demanda [MW]')
# ax.set_title('ARIMAX prediccion de tiempo de 15 minutos',style = "italic",fontsize=19)
ax.legend([r'$ P_{real} $',r'$ P_{prediccion} $'],ncol=2,loc='upper right',fontsize=15,fancybox=True, framealpha=0.5)
plt.tight_layout()

## Correlacion General

In [ ]:
corr_matrix = df.corr(method="pearson")

# Display the correlation matrix with a heatmap
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(35, 25)) 

sns.heatmap((corr_matrix),
            annot=True,
            annot_kws={'size': 6})  

In [ ]:
# Suponiendo que df es tu DataFrame
corr_matrix = df.corr(method="pearson")


first_row = corr_matrix.iloc[[0]]
fig, ax = plt.subplots(figsize=(40, 5))

sns.heatmap(abs(first_row),
            annot=True,
            annot_kws={'size': 14})  

plt.title('Correlación de las variables con respecto a la demanda actual') 
plt.show()

## Metricas de Evaluacion

## KPI

In [ ]:
def R2(y_true, y_pred):
    return r2_score(y_true, y_pred)

def ErrorAbsoluto(y_real,y_pred):
    return abs(y_real-y_pred)

def ErrorMaximo(y_true, y_pred):
    return max_error(y_true, y_pred)

def MAE(y_true, y_pred):
    return mean_absolute_error(y_true, y_pred)

def MSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred)

def MAPE(y_true, y_pred):
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mape

In [ ]:
def KPI(y_val,y_pred):
    R2_value = R2(y_val,y_pred)
    ErrorMax_value = ErrorMaximo(y_val,y_pred)
    MAE_value = MAE(y_val,y_pred)
    MSE_value = MSE(y_val,y_pred)
    MAPE_value = MAPE(y_val,y_pred)

    # Datos para la tabla
    data = [
        ["Métrica","R2","Error Max","MAE","MSE","MAPE"],
        ["Valor", f"{R2_value:.4f}",f"{ErrorMax_value:.4f}",f"{MAE_value:.4f}", f"{MSE_value:.4f}",f"{MAPE_value:.4f}%"]
    ]

    table_title = "Key Performance Indicator - KPI"
    table = tabulate(data, headers="firstrow", tablefmt="grid", numalign="center")
    title_centered = f"{' ' * ((len(table.splitlines()[0]) - len(table_title)) // 2)}{table_title}"
    print(title_centered)
    return print(table)

## Creacion de Graficos

In [ ]:
def FigGen(y_real,y_pred):
    tiempo = np.arange(0,len(y_pred),1)
    fig, ax = plt.subplots(1,1,figsize=(20, 10))
    plt.rcParams.update({'font.size': 23})

    # Set axis ranges; by default this will put major ticks every 25.
    ax.set_xlim(0,len(y_real[:168]))
    ax.set_ylim(min(y_pred),max(y_pred))

    # Change major ticks to show every 20.
    ax.xaxis.set_major_locator(MultipleLocator(200))
    ax.yaxis.set_major_locator(MultipleLocator(1000))

    # Change minor ticks to show every 5. (20/4 = 5)
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))

    # Turn grid on for both major and minor ticks and style minor slightly
    # differently.
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')

    ax.plot(tiempo,y_real,color='b')
    ax.plot(tiempo,y_pred,color='r',marker = "o", markersize = 5)
    ax.set_xlabel('Tiempo [Horas]\n')
    ax.set_ylabel('Demanda [MW]')
    # ax.set_title('ARIMAX prediccion de tiempo de 15 minutos',style = "italic",fontsize=19)
    ax.legend([r'$ D_{real} $',r'$ D_{prediccion} $'],ncol=2,loc='upper right',fontsize=15,fancybox=True, framealpha=0.5)
    plt.tight_layout()

## Error Absoluto

In [ ]:
def ErrorAbs(y_val,y_pred):
    z = ErrorAbsoluto(y_val,y_pred)
    tiempo = np.arange(0,len(z),1)/24

    fig, ax = plt.subplots(1,1,figsize=(20, 10))
    plt.rcParams.update({'font.size': 23})
    
    # Set axis ranges; by default this will put major ticks every 25.
    ax.set_xlim(-20,820)
    ax.set_ylim(0,3000)

    # Change major ticks to show every 20.
    ax.xaxis.set_major_locator(MultipleLocator(100))
    ax.yaxis.set_major_locator(MultipleLocator(500))

    # Change minor ticks to show every 5. (20/4 = 5)
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    ax.yaxis.set_minor_locator(AutoMinorLocator(5))

    # Turn grid on for both major and minor ticks and style minor slightly
    # differently.
    ax.grid(which='major', color='#CCCCCC', linestyle='--')
    ax.grid(which='minor', color='#CCCCCC', linestyle=':')

    ax.plot(tiempo,z,color='b')
    ax.set_xlabel('Tiempo [Horas]\n')
    ax.set_ylabel('Demanda [MW]')
    # ax.set_title('ARIMAX prediccion de tiempo de 15 minutos',style = "italic",fontsize=19)
    ax.legend([r'$ D_{errror} $'],ncol=2,loc='upper right',fontsize=15,fancybox=True, framealpha=0.5)
    return plt.tight_layout()

# Creacion de modelos

## Creacion de matrices

### Todas las variables

## Entrenamiento y Analisis de Modelo Estandar (Regresion Lineal)

In [ ]:
forecast = LinearRegression()

forecast.fit(DeM, Demanda_t0)
y_pred2 = forecast.predict(val_DeM)

In [ ]:
#Regresion Lineal

KPI(val_Demanda_t0,y_pred2)
FigGen(val_Demanda_t0,y_pred2)
FigEsp(val_Demanda_t0,y_pred2)
ErrorAbs(val_Demanda_t0,y_pred2)

# Informe de Modelos

In [ ]:
# Informe de Modelos